In [2]:
from pathlib import Path
from anndata import read_h5ad

import sys
import scanpy as sc
import os
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from scipy.stats import nbinom, pearsonr

from tqdm import tqdm

import pickle
import anndata as ad
from scipy.stats import zscore
from PIL import Image
from stimage._utils import gene_plot

In [1]:
from scipy import stats

def plot_correlation(df, attr_1, attr_2):
    r = stats.pearsonr(df[attr_1], 
                       df[attr_2])[0] **2

    g = sns.lmplot(data=df,
        x=attr_1, y=attr_2,
        height=5, legend=True
    )
    # g.set(ylim=(0, 360), xlim=(0,360))

    g.set_axis_labels(attr_1, attr_2)
    plt.annotate(r'$R^2:{0:.2f}$'.format(r),
                (max(df[attr_1])*0.9, max(df[attr_2])*0.9))
    return g


def calculate_correlation(attr_1, attr_2):
    r = stats.pearsonr(attr_1, 
                       attr_2)[0]
    return r

def calculate_correlation_2(attr_1, attr_2):
    r = stats.spearmanr(attr_1, 
                       attr_2)[0]
    return r

In [3]:
PATH_PRED = Path("/scratch/project_mnt/S0010/Xiao/Q1851/Xiao/Wiener_backup/STimage_exp/Skin_CODEX_pred")
# PATH_GC = Path("/scratch/imb/uqjxie6/benchmmarking/DeepHis2Exp/data/SkinVisium")
OUT_PATH = PATH_PRED / "PLOT"
OUT_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
adata_dict = {}
for file_path in PATH_PRED.glob("./*.h5ad"):
    library_id = file_path.stem.split("pred_")[-1]
    adata = ad.read_h5ad(file_path)
    adata_dict[library_id] = adata

In [5]:
adata_dict

{'S05_HE': AnnData object with n_obs × n_vars = 3965 × 1630
     obs: 'array_row', 'array_col', 'imagerow', 'imagecol', 'library_id', 'tile_tissue_mask_path', 'tissue_area', 'tile_path'
     uns: 'spatial'
     obsm: 'predicted_gene', 'spatial',
 'S02_HE': AnnData object with n_obs × n_vars = 6174 × 1630
     obs: 'array_row', 'array_col', 'imagerow', 'imagecol', 'library_id', 'tile_tissue_mask_path', 'tissue_area', 'tile_path'
     uns: 'spatial'
     obsm: 'predicted_gene', 'spatial'}

In [6]:
gene_list_CODEX = ['HLA-A', 'PTPRC', 'CD3E', 'CD4', 'FOXP3', 'GZMB', 'CD19', 'MS4A1', 'CR2', 'MPO', 'CD68', 'CD163', 'ITGAX', 'HLA-DRA', 'CD34', 'ACTA2', 'PDPN', 'KRT14', 'KRT8', 'KRT18', 'PMEL', 'SOX10', 'S100B', 'S100A4', 'HIF1A', 'VIM']

In [7]:
common_genes = list(set(gene_list_CODEX) & set(adata_dict["S05_HE"].var_names))
len(common_genes)

16

In [14]:
gene_to_plot = common_genes
for library_id, adata in adata_dict.items():
    adata = adata[:, common_genes]
    adata.uns['spatial'][library_id]['scalefactors'] = {'tissue_hires_scalef': 0.1}
    hires = Image.fromarray(adata.uns["spatial"][library_id]["images"]["fulres"])
    hires.thumbnail((hires.size[0]//10, hires.size[1]//10))
    adata.uns["spatial"][library_id]["images"]["hires"] = hires
    adata.X = adata.to_df().apply(zscore)
    for gene in gene_to_plot:
        gene_plot(adata, genes=gene, image_scale=True, library_id=library_id, spot_size=30,
                        output=str(OUT_PATH), name=f"pred_{library_id}_{gene}.pdf")
        plt.close()


In [8]:
# hires
gene_to_plot = common_genes
for library_id, adata in adata_dict.items():
    adata = adata[:, common_genes]
    adata.uns['spatial'][library_id]['scalefactors'] = {'tissue_hires_scalef': 1}
    hires = Image.fromarray(adata.uns["spatial"][library_id]["images"]["fulres"])
    hires.thumbnail((hires.size[0], hires.size[1]))
    adata.uns["spatial"][library_id]["images"]["hires"] = hires
    adata.X = adata.to_df().apply(zscore)
    for gene in gene_to_plot:
        fig = gene_plot(adata, genes=gene, image_scale=True, library_id=library_id, spot_size=5)
        fig.savefig(OUT_PATH / f"hires_pred_{library_id}_{gene}.png", dpi=2000)
        plt.close()

In [9]:
OUT_PATH

PosixPath('/scratch/project_mnt/S0010/Xiao/Q1851/Xiao/Wiener_backup/STimage_exp/Skin_CODEX_pred/PLOT')

In [1]:
# from stimage2.datasets.Xenium_dataset import Blake_xenium, Pfizer_xenium
# from pathlib import Path
# import pickle
# gene_list = []
# for i in range(len(sdataset)):
#     sdata = sdataset.load_data(i)
#     gene_list.append(sdata["table"].var_names.values.tolist())
# gene_list = list(set([item for items in gene_list for item in items]))
# with open(PATH_PRED / "xenium_gene_list.pkl", "wb") as f:
#     pickle.dump(gene_list, f)